# TODO:
https://www.kaggle.com/competitions/playground-series-s3e2/discussion/378795
https://www.kaggle.com/competitions/playground-series-s3e2/discussion/378780

These show that we should incorporate original data but when validating using K fold methods, we should only validate based on data in the competition dataset and not on original dataset. So implement this technique for this competition

### Also, bagging resulted in a lot better score with simple mean in the last competition, thought it didn't score much better on public, it absolutely ranked up to 60th position in the final private LB. So,
## Remember to trust your cvs over pbl

# A Few more TODOs:
* select features with less than or equal to 10 values as catergorical features, instead of current 20, see if it improves the score
* Try target encoding, weights of evidence AND leave one out encoding, see which one performs better

# Imports

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from pathlib import Path
import xgboost as xgb
import lightgbm as lgbm
import catboost
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
import optuna
from sklearn.preprocessing import StandardScaler

from category_encoders import TargetEncoder, LeaveOneOutEncoder, WOEEncoder

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Loading Data

In [4]:
BASE_PATH = Path('../input/playground-series-s3e3')

# id is not going to be an informative feature, so we're dropping it for train
# but since we'll need test set's ids to make the submission file, so we'll save those in  a separate varible before dropping
train = pd.read_csv(BASE_PATH / "train.csv").drop(columns="id")
test = pd.read_csv(BASE_PATH / "test.csv")
test_idx = test.id
test = test.drop(columns="id")

# It's been shown that incorporating original data, improves scores - at least on the public leaderboard. So let's do that!
original = pd.read_csv('../input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')

train.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,Gender,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,36,Travel_Frequently,599,Research & Development,24,3,Medical,1,4,Male,...,80,1,10,2,3,10,0,7,8,0
1,35,Travel_Rarely,921,Sales,8,3,Other,1,1,Male,...,80,1,4,3,3,4,2,0,3,0
2,32,Travel_Rarely,718,Sales,26,3,Marketing,1,3,Male,...,80,2,4,3,3,3,2,1,2,0
3,38,Travel_Rarely,1488,Research & Development,2,3,Medical,1,3,Female,...,80,0,15,1,1,6,0,0,2,0
4,50,Travel_Rarely,1017,Research & Development,5,4,Medical,1,2,Female,...,80,0,31,0,3,31,14,4,10,1


In [5]:
original.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


# Pre-Pre-Processing

### Let's make the feature names and order consistent b/w our competition dataset and original dataset, before we concatenate

In [6]:
original['Attrition'] = (original['Attrition'] == 'Yes').astype(np.int64)

# in original data, id is termed as "EmployeeNumber", so let's drop it
original.drop(columns="EmployeeNumber", inplace=True)

In [7]:
# now reordering the features in original dataset
original = original[list(train.columns)]

### Someone in the first compeition showed that adding a source feature i.e. a feature that indicates whether a given record is from original dataset or synthetic improves performance, so let's' do that!

In [8]:
original["is_original"] = 1
train["is_original"] = 0
test["is_original"] = 0

### Let's finally concatenate

In [9]:
train_extended = pd.concat([train, original]).reset_index(drop=True)
len(train_extended)

3147

### checking for null values

In [10]:
pd.concat([train_extended.isnull().sum().rename("Missing in Train"),
           test.isnull().sum().rename("Missing in Test")], axis=1).sort_values(by="Missing in Train")

,Missing in Train,Missing in Test
Age,0,0.0
Over18,0,0.0
OverTime,0,0.0
PercentSalaryHike,0,0.0
PerformanceRating,0,0.0
RelationshipSatisfaction,0,0.0
StandardHours,0,0.0
NumCompaniesWorked,0,0.0
StockOptionLevel,0,0.0
TrainingTimesLastYear,0,0.0


#### Insights: No missing values! Something to celebrate! :p

## Let's also concatenate test data to train

In [11]:
y = train_extended.Attrition
y

0       0
1       0
2       0
3       0
4       1
       ..
3142    0
3143    0
3144    0
3145    0
3146    0
Name: Attrition, Length: 3147, dtype: int64

In [12]:
df = pd.concat([train_extended.drop(columns="Attrition"), test])

# Preprocessing

### Identifying Categorical Features

In [13]:
df.dtypes.sort_values()

Age                          int64
YearsSinceLastPromotion      int64
YearsInCurrentRole           int64
YearsAtCompany               int64
WorkLifeBalance              int64
TrainingTimesLastYear        int64
TotalWorkingYears            int64
StockOptionLevel             int64
StandardHours                int64
RelationshipSatisfaction     int64
PerformanceRating            int64
PercentSalaryHike            int64
NumCompaniesWorked           int64
MonthlyRate                  int64
YearsWithCurrManager         int64
MonthlyIncome                int64
JobSatisfaction              int64
DailyRate                    int64
DistanceFromHome             int64
Education                    int64
EmployeeCount                int64
HourlyRate                   int64
EnvironmentSatisfaction      int64
JobLevel                     int64
JobInvolvement               int64
is_original                  int64
Gender                      object
MaritalStatus               object
OverTime            

### Remember, being of type int, doesn't mean that the feature cannot be categorial.
#### Let's check for unique values in each column

In [14]:
df.nunique().sort_values()

StandardHours                  1
EmployeeCount                  1
Over18                         1
is_original                    2
PerformanceRating              2
OverTime                       2
Gender                         2
BusinessTravel                 3
Department                     3
MaritalStatus                  3
RelationshipSatisfaction       4
JobSatisfaction                4
WorkLifeBalance                4
StockOptionLevel               5
JobInvolvement                 5
EnvironmentSatisfaction        5
Education                      6
JobLevel                       6
EducationField                 6
TrainingTimesLastYear          7
JobRole                        9
NumCompaniesWorked            11
PercentSalaryHike             15
YearsSinceLastPromotion       16
YearsWithCurrManager          18
YearsInCurrentRole            19
DistanceFromHome              29
YearsAtCompany                38
TotalWorkingYears             41
Age                           43
HourlyRate

#### INSIGHTS: Taking a quick look at number of unique values in features reveals that we should be safe setting the threshold for to 20 unique values for what consitutes as a categorical feature
#### We'll drop columns with only one value as they bring nothing to the table

#### But feel free to use your own intuition and test & trial to figure our what's works best in terms of threshold and features

In [15]:
feats_to_drop = [col for col in df.columns if df[col].nunique()==1]
cat_features = [col for col in df.columns if df[col].nunique() <= 20 and df[col].nunique() > 1]

In [16]:
df.drop(columns=feats_to_drop, inplace=True)

#### We won't use one hot encoder here, because we already have a large ratio of features to rows and one hotting would increase that ratio by a large margin even further which will result in severe overfitting
#### Rather we'll use ordinal/label encoder (they're basically the same thing)

In [17]:
len(df), len(y)

(4266, 3147)

In [18]:
# but first let's separate test and train_extended
X_train = df.iloc[:-len(test), :]
X_test = df.iloc[-len(test): , :]

In [19]:
target_enc = TargetEncoder()
loo_enc = LeaveOneOutEncoder(sigma=0.05)
woe_enc = WOEEncoder(sigma=0.05)

target_enc.fit(X_train[cat_features], y)

X_train[cat_features] = target_enc.transform(X_train[cat_features], y)
X_test[cat_features] = target_enc.transform(X_test[cat_features])

X_train.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,...,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,is_original
0,36,0.223048,599,0.121711,24,3,0.124383,4,0.147996,42,...,2,1,10,2,3,10,0,7,8,0
1,35,0.128590,921,0.173391,8,3,0.103659,1,0.147996,46,...,4,1,4,3,3,4,2,0,3,0
2,32,0.128590,718,0.173391,26,3,0.196141,3,0.147996,80,...,4,2,4,3,3,3,2,1,2,0
3,38,0.128590,1488,0.121711,2,3,0.124383,3,0.124063,40,...,3,0,15,1,1,6,0,0,2,0
4,50,0.128590,1017,0.121711,5,4,0.124383,2,0.124063,37,...,3,0,31,0,3,31,14,4,10,0


In [20]:
# lets glue them back together
df = pd.concat([X_train, X_test])
len(df)

4266

## Always a good idea to scale the features

In [21]:
# we don't want to scale the source column
sc = StandardScaler()
df_scaled = df.copy()
df_scaled.iloc[:, :-1] = pd.DataFrame(sc.fit_transform(df.iloc[:, :-1]), columns=df.columns[:-1])
df_scaled.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,...,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,is_original
0,-0.042187,1.944557,-0.676304,-0.681781,1.914406,0.067799,-0.490679,1.151976,0.786359,-1.276573,...,-0.680038,0.299291,-0.120890,-0.638312,0.351622,0.512223,-1.167739,1.553534,1.052484,0
1,-0.155733,-0.245174,0.160523,1.402552,-0.108063,0.067799,-1.189936,-1.590395,0.786359,-1.074371,...,1.182590,0.299291,-0.927119,0.188354,0.351622,-0.489294,-0.609548,-0.655171,-0.331151,0
2,-0.496370,-0.245174,-0.367042,1.402552,2.167215,0.067799,1.930496,0.237852,0.786359,0.644348,...,1.182590,1.526962,-0.927119,0.188354,0.351622,-0.656214,-0.609548,-0.339641,-0.607878,0
3,0.184905,-0.245174,1.634065,-0.681781,-0.866489,0.067799,-0.490679,0.237852,-1.271683,-1.377674,...,0.251276,-0.928379,0.550967,-1.464978,-2.630526,-0.155455,-1.167739,-0.655171,-0.607878,0
4,1.547454,-0.245174,0.410011,-0.681781,-0.487276,1.041633,-0.490679,-0.676271,-1.271683,-1.529326,...,0.251276,-0.928379,2.700910,-2.291644,0.351622,4.017536,2.739597,0.606946,1.605938,0


### Let's seprate test and train sets

In [22]:
X_train = df_scaled.iloc[:-len(test), :]
X_test = df_scaled.iloc[-len(test): , :]

# Modelling

### But first, let's setup cross validation

In [23]:
# for i, (x, y) in enumerate(zip([1,2,3], [4,5,6])):
#     print(f"{'*'*10} {i}")
#     print(f"X: {x}")
#     print(f"Y: {y}")    

In [24]:
# a = np.array([1,2,3])
# b = np.array([4,5,6])

# np.append(a, b)

In [25]:
# we're gonna train on the combined dataset but, we'll only calculate the validation score only on comp data

# N_FOLDS = 10

def cross_validate(X, y, model, model_verbose=None, verbose=None):
    N_FOLDS = 5

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=1337)
    skf_2 = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=369)
    all_scores = []
    
    X_comp = X[X.is_original==0]
    y_comp = y[X_comp.index]
    X_original = X[X.is_original==1]
    y_original = y[X_original.index]
    
    # the k fold splitter created indcies based on the length from 0 to len-1, it doesn't take into account the index as I thought.
    # so we reset the index to match with the indices generated by splitter to access those elements
    X_original = X_original.reset_index(drop=True)
    y_original = y_original.reset_index(drop=True)
    
    for fold_id, ((train_comp_idx, val_comp_idx), (train_org_idx, val_org_idx)) \
                in enumerate(zip(skf.split(X_comp, y_comp), skf_2.split(X_original, y_original))):
        
        
        # for training we'll use data from both datasets
        X_tr_comp = X_comp.iloc[train_comp_idx]
        X_tr_org = X_original.iloc[train_org_idx]
        
        y_tr_comp = y_comp[train_comp_idx]
        y_tr_org = y_original[train_org_idx]
        
        
        # lets combine both
        X_tr = pd.concat([X_tr_comp, X_tr_org], axis=0).reset_index(drop=True)
        y_tr = pd.concat([y_tr_comp, y_tr_org], axis=0).reset_index(drop=True)
        
        
        # but for validation we'll use only competition data        
        X_val = X_comp.iloc[val_comp_idx]
        y_val = y_comp[val_comp_idx]
               
        model.fit(X_tr, y_tr, 
                  eval_set=[(X_val, y_val)],
                  early_stopping_rounds=50,
                 verbose=model_verbose)
        
        y_pred = model.predict_proba(X_val)[:, 1]
        
        auc = roc_auc_score(y_val, y_pred)
        
        print(f"Fold {fold_id} \t auc: {auc}")
        
        all_scores.append(auc)
    
    avg_auc = np.mean(all_scores)
    
    print(f"Avg AUC: {avg_auc}")

In [26]:
# random params values - make sure to tune yours
xgb_params = {'n_estimators': 150,
                 'max_depth': 3,
                 'learning_rate': 0.1,
                 'min_child_weight': 4,
                 'subsample': 0.7,
                 'colsample_bytree': 0.3
             }


xgb_clf = xgb.XGBClassifier(**xgb_params)

cross_validate(X_train, y, xgb_clf, model_verbose=False)

# xgb_clf.fit(X_train, y, verbose=0)

Fold 0 	 auc: 0.8373310810810811
Fold 1 	 auc: 0.9072635135135134
Fold 2 	 auc: 0.8171186440677966
Fold 3 	 auc: 0.8738983050847458
Fold 4 	 auc: 0.9008474576271186
Avg AUC: 0.867291800274851


In [ ]:
# random params but feel free to tune
lgbm_params = {'n_estimators': 407,
                 'num_rounds': 274,
                 'learning_rate': 0.1,
                 'num_leaves': 195,
                 'max_depth': 9,
                 'min_data_in_leaf': 46,
                 'lambda_l1': 0.01,
                 'lambda_l2': 0.6,
                 'min_gain_to_split': 1.42,
                 'bagging_fraction': 0.45,
                 'feature_fraction': 0.3}


lgbm_clf = lgbm.LGBMClassifier(**lgbm_params)

cross_validate(X_train, y, lgbm_clf, model_verbose=-1)

# lgbm_clf.fit(X_train, y, verbose=False)

In [50]:
#random params but feel free to tune
catboost_params = {'loss_function': 'CrossEntropy',
                     'learning_rate': 0.76,
                     'l2_leaf_reg': 0.014,
                     'colsample_bylevel': 0.06,
                     'depth': 1,
                     'boosting_type': 'Plain',
                     'bootstrap_type': 'Bernoulli',
                     'min_data_in_leaf': 18,
                     'one_hot_max_size': 14,
                     'subsample': 0.99}

catboost_clf = catboost.CatBoostClassifier(**catboost_params)

cross_validate(X_train, y, catboost_clf, model_verbose=False)

# catboost_clf.fit(X_train, y, verbose=False)

Fold 0 	 auc: 0.8514358108108109
Fold 1 	 auc: 0.9086993243243243
Fold 2 	 auc: 0.8044915254237287
Fold 3 	 auc: 0.8661864406779661
Fold 4 	 auc: 0.9154237288135594
Avg AUC: 0.8692473660100777


## INSIGHTS:
let's use this method of cross validation to
* Tune all our models
* Select top k
* Take their predictions average
* submit

In [75]:
np.random.randint(1, 10, size=(2,3))

array([[7, 2, 9],
       [7, 1, 7]])

In [83]:
some_X = pd.DataFrame(data=np.random.randint(1, 10, size=(2,3)))
pd.concat([some_X, some_X], axis=0).reset_index(drop=True)

,0,1,2
0,6,1,9
1,3,3,5
2,6,1,9
3,3,3,5


# Hyperparameters Tuning

## XGBoost

In [31]:
def objective_xgb(trial, X, y):
    params = {
        "tree_method": "gpu_hist",
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 0.00001, 0.3),
        'subsample': trial.suggest_loguniform('subsample', 0.2, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.2, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.00001, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.00001, 1.0),
        'early_stopping_rounds': trial.suggest_int("early_stoppig_rounds", 40, 100)
    }
    

    # we're gonna train on the combined dataset but, we'll only calculate the validation score only on comp data

    N_FOLDS = 5

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=1337)
    skf_2 = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=369)
    all_scores = []
    
    X_comp = X[X.is_original==0]
    y_comp = y[X_comp.index]
    X_original = X[X.is_original==1]
    y_original = y[X_original.index]
    
    # the k fold splitter created indcies based on the length from 0 to len-1, it doesn't take into account the index as I thought.
    # so we reset the index to match with the indices generated by splitter to access those elements
    X_original = X_original.reset_index(drop=True)
    y_original = y_original.reset_index(drop=True)
    
    for fold_id, ((train_comp_idx, val_comp_idx), (train_org_idx, val_org_idx)) \
                in enumerate(zip(skf.split(X_comp, y_comp), skf_2.split(X_original, y_original))):
        
        
        # for training we'll use data from both datasets
        X_tr_comp = X_comp.iloc[train_comp_idx]
        X_tr_org = X_original.iloc[train_org_idx]
        
        y_tr_comp = y_comp[train_comp_idx]
        y_tr_org = y_original[train_org_idx]
        
        
        # lets combine both
        X_tr = pd.concat([X_tr_comp, X_tr_org], axis=0).reset_index(drop=True)
        y_tr = pd.concat([y_tr_comp, y_tr_org], axis=0).reset_index(drop=True)
        
        
        # but for validation we'll use only competition data        
        X_val = X_comp.iloc[val_comp_idx]
        y_val = y_comp[val_comp_idx]
        
        model = xgb.XGBClassifier(**params)
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
        
        y_pred = model.predict_proba(X_val)[:, 1]
        
        # for some reaason the score isn't even touch 90, but on pbL we're on 94, really weird. Let's check how much we score on original data
        
        auc = roc_auc_score(y_val, y_pred)
        
#         print(f"Fold {fold_id} \t auc: {auc}")
        
        all_scores.append(auc)
    
    avg_auc = np.mean(all_scores)
    
    print(f"Avg AUC: {avg_auc}")
    
    return avg_auc

In [32]:
study_xgb = optuna.create_study(study_name="xgboost_tuning", direction="maximize")
func = lambda trial: objective_xgb(trial, X_train, y)
study_xgb.optimize(func, n_trials=100)

[I 2023-01-21 07:19:16,616] A new study created in memory with name: xgboost_tuning
[I 2023-01-21 07:19:17,513] Trial 0 finished with value: 0.8514896358222629 and parameters: {'n_estimators': 292, 'max_depth': 7, 'learning_rate': 0.2599300901450483, 'min_child_weight': 7, 'gamma': 0.0006123196326410788, 'subsample': 0.5555352165233362, 'colsample_bytree': 0.5232475882135765, 'reg_alpha': 0.03767451568087707, 'reg_lambda': 6.591529774651591e-05, 'early_stoppig_rounds': 43}. Best is trial 0 with value: 0.8514896358222629.


Avg AUC: 0.8514896358222629


[I 2023-01-21 07:19:19,333] Trial 1 finished with value: 0.8618841044434264 and parameters: {'n_estimators': 437, 'max_depth': 4, 'learning_rate': 0.08480202209492158, 'min_child_weight': 7, 'gamma': 0.013796760877067646, 'subsample': 0.35234481267712675, 'colsample_bytree': 0.594228629283936, 'reg_alpha': 0.10673960618116766, 'reg_lambda': 1.1299768215765113e-05, 'early_stoppig_rounds': 77}. Best is trial 1 with value: 0.8618841044434264.


Avg AUC: 0.8618841044434264


[I 2023-01-21 07:19:21,629] Trial 2 finished with value: 0.8583121850664224 and parameters: {'n_estimators': 383, 'max_depth': 4, 'learning_rate': 0.04631304014448371, 'min_child_weight': 7, 'gamma': 0.1228425078609511, 'subsample': 0.3853975150208087, 'colsample_bytree': 0.21364246957667102, 'reg_alpha': 0.2735838325858485, 'reg_lambda': 0.4090330296468641, 'early_stoppig_rounds': 69}. Best is trial 1 with value: 0.8618841044434264.


...

Avg AUC: 0.8225634448007328


[I 2023-01-21 07:22:31,416] Trial 98 finished with value: 0.8654058634906093 and parameters: {'n_estimators': 337, 'max_depth': 2, 'learning_rate': 0.11640322172399498, 'min_child_weight': 1, 'gamma': 0.005130693116532587, 'subsample': 0.2409058772390851, 'colsample_bytree': 0.7081342777861455, 'reg_alpha': 0.00014354952365015287, 'reg_lambda': 0.19127346965464373, 'early_stoppig_rounds': 42}. Best is trial 96 with value: 0.872599289967934.


Avg AUC: 0.8654058634906093


[I 2023-01-21 07:22:33,327] Trial 99 finished with value: 0.8431724118185983 and parameters: {'n_estimators': 464, 'max_depth': 7, 'learning_rate': 0.09408342271521741, 'min_child_weight': 1, 'gamma': 0.00037365513657391193, 'subsample': 0.20479026479429752, 'colsample_bytree': 0.6243860182113483, 'reg_alpha': 1.9994784604069627e-05, 'reg_lambda': 0.0012711225541654138, 'early_stoppig_rounds': 48}. Best is trial 96 with value: 0.872599289967934.


Avg AUC: 0.8431724118185983


In [33]:
study_xgb.best_value

0.872599289967934

## INSIGHTS:
* With 10 fold cv and target encoding, we achieve a best value of 0.879. But i feel like 10 cv might be too much for such small dataset in that it's way more prone to randoness than a 5 fold is
* With 5 fold cv and target encoding, we achieve a best value of 0.87259 or 0.873

Although the pbL is just luck based casino game at this point, we'll still submit using the best params to make sure we're on the right path with such cross_validation technique. The last time i tried using this technique, it resulted in severe overfitting because idk how i somehow messed something up

In [34]:
study_xgb.best_params

{'n_estimators': 461,
 'max_depth': 2,
 'learning_rate': 0.11634909429778409,
 'min_child_weight': 1,
 'gamma': 0.00036770696205157646,
 'subsample': 0.21037865602062264,
 'colsample_bytree': 0.607771518151508,
 'reg_alpha': 5.279597083207242e-05,
 'reg_lambda': 0.5386506165822396,
 'early_stoppig_rounds': 43}

In [36]:
# # parameters that overfit seeverelyy
# xgb_params = {'n_estimators': 462,
#                  'max_depth': 9,
#                  'learning_rate': 0.24754839357747555,
#                  'min_child_weight': 1,
#                  'gamma': 0.0002768382878439927,
#                  'subsample': 0.4962635986679422,
#                  'colsample_bytree': 0.43112487341982364,
#                  'reg_alpha': 2.4326609397287665e-05,
#                  'reg_lambda': 0.21844284657376126,
#                  'early_stoppig_rounds': 45}

# new shiny params promising to not overfit
xgb_params = {'n_estimators': 461,
                 'max_depth': 2,
                 'learning_rate': 0.11634909429778409,
                 'min_child_weight': 1,
                 'gamma': 0.00036770696205157646,
                 'subsample': 0.21037865602062264,
                 'colsample_bytree': 0.607771518151508,
                 'reg_alpha': 5.279597083207242e-05,
                 'reg_lambda': 0.5386506165822396,
                 'early_stoppig_rounds': 43}

In [37]:
X_train_fr, X_val, y_train_fr, y_val = train_test_split(X_train, y, test_size=0.1, shuffle=True, random_state=1337,
                                                        stratify=y)

In [38]:
xgb_tuned_clf = xgb.XGBClassifier(**xgb_params)
xgb_tuned_clf.fit(X_train_fr, y_train_fr, eval_set=[(X_val, y_val)], verbose=False)

[07:26:28] WARNING: ../src/learner.cc:627: 
Parameters: { "early_stoppig_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.607771518151508, early_stoppig_rounds=43,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.00036770696205157646, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.11634909429778409,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=2,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=461, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=5.279597083207242e-05, ...)

In [39]:
xgb_tuned_preds = xgb_tuned_clf.predict_proba(X_test)[:, 1]

In [41]:
# non-overfitting predictions
submission = pd.DataFrame({"id": test_idx, "Attrition": xgb_tuned_preds})
submission.head()

,id,Attrition
0,1677,0.401170
1,1678,0.117693
2,1679,0.000777
3,1680,0.015711
4,1681,0.295577


In [42]:
# OVERFITTED PREDICTIONS
# submission = pd.DataFrame({"id": test_idx, "Attrition": xgb_tuned_preds})
# submission.head()

,id,Attrition
0,1677,0.021201
1,1678,0.001126
2,1679,0.000017
3,1680,0.000405
4,1681,0.947035


In [42]:
submission.to_csv("submission.csv", index=False)